In [1]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv("agg.csv")
data = data.iloc[:,1:]
data.columns = ["state", "prod", "n"]
products = list(data.groupby("prod"))

In [12]:
mydict = {}
paesi = {}
for prodotto in products:
    tab = prodotto[1]
    for i in list(range(0,len(tab))):
        paese = tab.iloc[i].state
        if paese in list(paesi.keys()):
            paesi[paese]+=tab.iloc[i].n
        else:
            paesi[paese]=tab.iloc[i].n
        if len(tab)>1:    
            for j in list(range(i+1,len(tab))):
                chiave = tab.iloc[i].state + "\t" + tab.iloc[j].state
                chiave2 = tab.iloc[j].state + "\t" + tab.iloc[i].state
                valore = [tab.iloc[i].n, tab.iloc[j].n]
                if chiave in list(mydict.keys()):
                    mydict[chiave][0] += valore[0]
                    mydict[chiave][1] += valore[1]
                elif chiave2 in list(mydict.keys()):
                    mydict[chiave2][0] += valore[0]
                    mydict[chiave2][1] += valore[1]
                else:
                    mydict[chiave] = valore

In [15]:
for el in mydict:
    lista = el.split("\t")
    totale = paesi[lista[0]] * paesi[lista[1]]
    mydict[el] = (mydict[el][0] * mydict[el][1]) / totale

In [50]:
f = open("edges.csv", "w")
f.write("Source" + "\t" + "Target" + "\t" + "Weight" + "\n")
for el in mydict:
    w = round(mydict[el], 2)
    if w >= 0.25:
        f.write(el + "\t" + str(w) + "\n")
f.close()

In [72]:
edges = pd.read_csv("edges.csv", sep='\t')
nodi = pd.read_csv("nodi.csv")
edges = edges[((edges.Source!="Unspecified") & (edges.Target!="Unspecified"))]
def sino(x):
    if ((x in list(edges.Source)) | (x in list(edges.Target))):
        return(True)
    else:
        return(False)
sel = nodi.Id.apply(sino)
nodi[sel].to_csv("nodi.csv", index=False)
edges.to_csv("edges.csv", index=False)